[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fga-0/automating_data_augmentation/blob/main/wrn_raw.ipynb)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim

import utils
import importlib
importlib.reload(utils)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_to_data = "drive/MyDrive/SDD/data_augmentation/cifar10"

In [ ]:
# Define transformations (normalization, conversion to tensor)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
    # transforms.Normalize((0.5,), (0.5,))
])

# Load full CIFAR-10 dataset
# # Les images CIFAR10 sont de dimension 32x32.
trainset = torchvision.datasets.CIFAR10(root=path_to_data, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=path_to_data, train=False, download=True, transform=transform)

In [ ]:
from utils import get_trainloader_valloader

In [ ]:
train_loader, val_loader = get_trainloader_valloader(dataset=trainset,
                                                     split=0.8,
                                                     batch_size=128
                                                     )
test_loader = DataLoader(testset, batch_size=128, shuffle=False)

In [ ]:
from torchvision.models import wide_resnet50_2

# Load Wide ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Wide ResNet-50-2 model:
wrn = wide_resnet50_2(pretrained=True)
# Freeze all model parameters except for the final layer:
for param in wrn.parameters():
    param.requires_grad = False
# Get the number of input features for the original last layer:
num_feature = wrn.fc.in_features
# Replace the final classification layer to match your dataset:
wrn.fc = nn.Linear(num_feature, 10)
# View the structure of the new final layer (optional):
print(wrn.fc)
# Move the model to the GPU for accelerated training:
wrn = wrn.to(device)

print(f"Using device: {device}")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(wrn.fc.parameters(), lr=0.001)


# Learning rate scheduler for better convergence
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
patience = 3
min_delta_loss = 0.001
min_delta_accuracy = 0.1

In [ ]:
wrn_trained = utils.train_WideResNet(model=wrn, trainloader=train_loader, valloader=val_loader, num_epochs=50, 
                                     batch_size=128, optimizer=optimizer, criterion=criterion, 
                                     device=device, scheduler=scheduler, patience=patience, 
                                     min_delta_loss=min_delta_loss, min_delta_accuracy=min_delta_accuracy)

In [ ]:
utils.evaluate(wrn_trained, test_loader, device)